In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

In [2]:
#import training datadet
#import internal data
inter = pd.ExcelFile('training data_Github.xlsx')
inter = pd.read_excel(inter,'Combined')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
#print(comb.info())

In [4]:
#READ DATA

y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
X = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)
#X['type'] = 'train'
X_wo = X.drop(['H','S*','A','B','C (ph=2.8)','C (ph=7.0)','EB retention factor','particle size (um)','length (mm)','diameter (mm)','flow rate (mL/min)','PH'], axis=1)
#X_wo['type'] = 'train'

print(X.info())
print(X_wo.info())
print(y.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 129 entries, MaxAEI to fsulfone
dtypes: float64(90), int64(39)
memory usage: 1.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tR (min)  1790 non-null   float64
dtypes: float64(1)
memory usage: 14.1 KB
None


In [5]:
#ANN Regression with SP and MP


### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference 
X_scaled=PredictorScaler.fit_transform(X)
y_scaled=TargetVarScaler.fit_transform(y)

# Generating the standardized values of X and y 
#X=PredictorScalerFit.transform(X)
#y_scale=TargetVarScalerFit.transform(y)
X_test_scaled=PredictorScaler.transform(X)
              
#X = pd.DataFrame(X)
y_scaled = pd.DataFrame(y_scaled)
X_scaled = pd.DataFrame(X_scaled)

#Convert from dataframe to array
y_train = np.asarray(y_scaled).astype('float32')
X_train = np.asarray(X_scaled).astype('float32')               


# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# Defining the Third layer of the model
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')    
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 50, epochs = 50, verbose=0)

# Generating Predictions on testing data
Predictions=model.predict(X_test_scaled)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScaler.inverse_transform(Predictions)

results_ANN = pd.DataFrame(Predictions,columns = ['pred tR by ANN (min)'])
measured_y = pd.DataFrame(np.asarray(y).astype('float32'),columns = ['measured tR (min)'])
results_ANN = pd.concat([measured_y,results_ANN],axis=1)

print(results_ANN.info())
results_ANN.to_csv('results_ANN.csv')

56/56 [==============================] - 0s 2ms/step
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   measured tR (min)     1790 non-null   float32
 1   pred tR by ANN (min)  1790 non-null   float32
dtypes: float32(2)
memory usage: 14.1 KB
None


In [6]:

#ANN Regression without SP and MP


### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
XScaler=StandardScaler()
YScaler=StandardScaler()

# Storing the fit object for later reference
X_wo_scaled=XScaler.fit_transform(X_wo)
y_scaled=YScaler.fit_transform(y)

# Generating the standardized values of X and y
#X=PredictorScalerFit.transform(X)
#y=TargetVarScalerFit.transform(y)
X_wo_test_scaled=XScaler.transform(X_wo)
               
#X = pd.DataFrame(X)
y_scaled = pd.DataFrame(y_scaled)
X_wo_scaled = pd.DataFrame(X_wo_scaled)

#Convert from dataframe to array

y_train = np.asarray(y_scaled).astype('float32')
X_wo_train = np.asarray(X_wo_scaled).astype('float32')               


# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=129, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# Defining the Third layer of the model
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')    
# Fitting the ANN to the Training set
model.fit(X_wo_train, y_train ,batch_size = 50, epochs = 50, verbose=0)

# Generating Predictions on testing data
Predictions=model.predict(X_wo_test_scaled)

# Scaling the predicted Price data back to original price scale
Predictions=YScaler.inverse_transform(Predictions)

results_ANN = pd.DataFrame(Predictions,columns = ['pred tR by ANN (min)'])
measured_y = pd.DataFrame(np.asarray(y).astype('float32'),columns = ['measured tR (min)'])
results_ANN = pd.concat([measured_y,results_ANN],axis=1)

print(results_ANN.info())
results_ANN.to_csv('results_ANN_wo.csv')


56/56 [==============================] - 0s 1ms/step
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   measured tR (min)     1790 non-null   float32
 1   pred tR by ANN (min)  1790 non-null   float32
dtypes: float32(2)
memory usage: 14.1 KB
None
